In [16]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [17]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR.jl")
include("./libs/MMR-W.jl")

MMR_W (generic function with 1 method)

In [18]:
# 評価基準数
N = 6;
# 真の区間重要度のセッティング
true_importance = "D"; # ["A", "B", "C", "D", "E"]

In [19]:
# 真の区間重要度を読み込み
df = CSV.File("./data/N"*string(N)*"_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:N
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

display(L"T = %$(intervalVectorLaTeXString(T))")

L"$T = \begin{pmatrix} \left[ 0.21, 0.25 \right] \\ \left[ 0.17, 0.23 \right] \\ \left[ 0.14, 0.22 \right] \\ \left[ 0.1, 0.2 \right] \\ \left[ 0.07, 0.19 \right] \\ \left[ 0.04, 0.18 \right] \end{pmatrix}$"

In [20]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./data/N"*string(N)*"_a3_"*true_importance*"_PCM_int.csv", header=false) |> DataFrame

# データフレームをN×Nのサブデータフレームに分割
subdfs = split_dataframe(df1, N);

In [21]:
# 結果格納用

# 1000個のデータで推定
result_array = []
for (i, subdf) in enumerate(subdfs)

    # 一対比較行列を取り出す
    A = Matrix(subdf)

    # 提案手法で区間重要度推定
    # solution = MMR(A, EV)
    solution = MMR_W(A, EV)  
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    deviation = (inf.(E).+sup.(E))./2 - (inf.(T).+sup.(T))./2
    result_array = vcat(result_array, deviation)
end


In [22]:
sum_of_squares = sum(result_array .^ 2)

4.280077700040106